In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import sys

import torch.nn.functional as F

In [2]:
############## TENSORBOARD ########################
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/mnist")
###################################################

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Hyper-parameters 
input_size = 784 # 28x28
hidden_size = 500 
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [5]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [6]:
examples = iter(test_loader)
example_data, example_targets = examples.next()

# for i in range(6):
#     plt.subplot(2,3,i+1)
#     plt.imshow(example_data[i][0], cmap='gray')
# plt.show()

############## TENSORBOARD ########################
img_grid = torchvision.utils.make_grid(example_data)
writer.add_image('mnist_images', img_grid)
writer.close()
#sys.exit()
###################################################

In [7]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end # loss - crossent
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

############## TENSORBOARD ########################
writer.add_graph(model, example_data.reshape(-1, 28*28).to(device)) # .to(device) 안해서 오류남
# writer.close()
# sys.exit() # jupyter에서는 cell 별로 실행시킬 수 있는데, 
             #vscode 에서 돌릴 때 중간에 멈추고 싶은 경우
###################################################

In [9]:
# Train the model
n_total_steps = len(train_loader)

############## TENSORBOARD ########################
running_loss = 0.0
running_correct = 0
###################################################

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        running_correct += (predicted == labels).sum().item()
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], \
            Loss: {loss.item():.4f}')
            ############## TENSORBOARD ########################
            writer.add_scalar('trainig loss', running_loss / 100, \
                              epoch * n_total_steps + i)
            writer.add_scalar('accuracy', running_correct / 100, \
                              epoch * n_total_steps + i)
            running_loss = 0.0
            running_correct = 0
            ###################################################
            

Epoch [1/2], Step [100/600],             Loss: 0.4147
Epoch [1/2], Step [200/600],             Loss: 0.2439
Epoch [1/2], Step [300/600],             Loss: 0.1838
Epoch [1/2], Step [400/600],             Loss: 0.2513
Epoch [1/2], Step [500/600],             Loss: 0.1686
Epoch [1/2], Step [600/600],             Loss: 0.2591
Epoch [2/2], Step [100/600],             Loss: 0.1287
Epoch [2/2], Step [200/600],             Loss: 0.0867
Epoch [2/2], Step [300/600],             Loss: 0.1761
Epoch [2/2], Step [400/600],             Loss: 0.1597
Epoch [2/2], Step [500/600],             Loss: 0.0579
Epoch [2/2], Step [600/600],             Loss: 0.0911


In [10]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
class_labels = []
class_preds = []
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        values, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        class_probs_batch = [F.softmax(output, dim=0) for output in outputs]

        class_preds.append(class_probs_batch)
        class_labels.append(predicted)

    # 10000, 10, and 10000, 1
    # stack concatenates tensors along a new dimension
    # cat concatenates tensors in the given dimension
    class_preds = torch.cat([torch.stack(batch) for batch in class_preds])
    class_labels = torch.cat(class_labels)

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

    ############## TENSORBOARD ########################
    classes = range(10)
    for i in classes:
        labels_i = class_labels == i
        preds_i = class_preds[:, i]
        writer.add_pr_curve(str(i), labels_i, preds_i, global_step=0)
        writer.close()
    ###################################################

Accuracy of the network on the 10000 test images: 97.08 %


확률을 output으로 보려면 softmax가 필요

add_pr_curve   

PR curves: precision   

How many TP, TN, FP, FN